In [293]:
with open('19a_test.txt') as f:
    lines = f.readlines()

lines = [line.strip() for line in lines]

In [294]:
from typing import Tuple
import itertools
from math import copysign

In [295]:
def rotation_sign(s: str) -> int:
    if (s[0]) == '-': return -1
    return 1

In [296]:
def rotate_transform(b_pos: Tuple[int, int, int], orientation: str) -> Tuple[int, int, int]:
    orientation1 = orientation.split(',')
    if orientation1[0] == 'x' or orientation1[0] == '-x': xi1 = 0
    elif orientation1[0] == 'y' or orientation1[0] == '-y': yi1 = 0
    elif orientation1[0] == 'z' or orientation1[0] =='-z': zi1 = 0
    else: print("o0")
    
    if orientation1[1] == 'x' or orientation1[1] == '-x': xi1 = 1
    elif orientation1[1] == 'y' or orientation1[1] == '-y': yi1 = 1
    elif orientation1[1] == 'z' or orientation1[1] == '-z': zi1 = 1
    else: print('o1')

    if orientation1[2] == 'x' or orientation1[2] == '-x': xi1 = 2
    elif orientation1[2] == 'y' or orientation1[2] == '-y': yi1 = 2
    elif orientation1[2] == 'z' or orientation1[2] == '-z': zi1 = 2
    else: print('o2')

    x1sgn = -1 if orientation1[xi1][0] == '-' else 1
    y1sgn = -1 if orientation1[yi1][0] == '-' else 1
    z1sgn = -1 if orientation1[zi1][0] == '-' else 1

    return (b_pos[xi1]*x1sgn, b_pos[yi1]*y1sgn, b_pos[zi1]*z1sgn)
    

In [297]:
def same_beacons(a1: Tuple[int, int, int], a2: Tuple[int, int, int], b1: Tuple[int, int, int], b2: Tuple[int, int, int], rotation: str) -> bool: 
        i_coord, j_coord, k_coord = rotation.split(',')
        x_diff = a1[0] - a2[0]
        y_diff = a1[1] - a2[1]
        z_diff = a1[2] - a2[2]

        i_diff = (b1[0] - b2[0]) * rotation_sign(i_coord)
        j_diff = (b1[1] - b2[1]) * rotation_sign(j_coord)
        k_diff = (b1[2] - b2[2]) * rotation_sign(k_coord)

        if i_coord[-1] == 'x': x_comp = x_diff == i_diff
        elif i_coord[-1] == 'y': x_comp = y_diff == i_diff
        elif i_coord[-1] == 'z': x_comp = z_diff == i_diff
        else: print('i-coord error.')

        if j_coord[-1] == 'x': y_comp = x_diff == j_diff
        elif j_coord[-1] == 'y': y_comp = y_diff == j_diff
        elif j_coord[-1] == 'z': y_comp = z_diff == j_diff
        else: print('j-coord error.')

        if k_coord[-1] == 'x': z_comp = x_diff == k_diff
        elif k_coord[-1] == 'y': z_comp = y_diff == k_diff
        elif k_coord[-1] == 'z': z_comp = z_diff == k_diff
        else: print('k-coord error.')

        return x_comp and y_comp and z_comp

In [298]:
def find_commons(rotations, lb1, lb2, or1):
    for rotation in rotations:
        common_beacons = []
        other_common_beacons = []
        for i in range(len(lb1)):
            if len(common_beacons) >= 12: print("hi")
            a1 = lb1[i]
            for j in range(i+1,len(lb1)):
                # if len(common_beacons) >= 12: break
                a2 = lb1[j]
                for k in range(len(lb2)):
                    # if len(common_beacons) >= 12: break
                    b1 = rotate_transform(lb2[k], rotation)
                    for l in range(k+1, len(lb2)):
                        b2 = rotate_transform(lb2[l], rotation)
                        # same: whether or not these two pairs of beacons have the same magnitude of difference.
                        same = a1[0]-a2[0] == b1[0]-b2[0] and a1[1]-a2[1] == b1[1]-b2[1] and a1[2]-a2[2] == b1[2]-b2[2]
                        if same:
                            if i not in common_beacons: common_beacons.append(i)
                            if j not in common_beacons: common_beacons.append(j)
                            if k not in other_common_beacons: other_common_beacons.append(k)
                            if l not in other_common_beacons: other_common_beacons.append(l)
                            if len(common_beacons) >= 12:
                                
                                # figure out the absolute rotation right now ??
                                # '-y x -z' base
                                # 'x z -y' curr
                                # what is the absolute rotation?
                                # [0] value -> -y
                                # [1] value -> -z
                                # [2] value -> -x
                                # super_rotation = or1
                                # s0, s1, s2 = super_rotation.split(',')
                                # zero,one,two = rotation.split(',')

                                # if zero == 'x': good_x = s0
                                # elif zero == '-x':
                                #     if s0[0] == '-': good_x = s0[-1]
                                #     else: good_x = '-'+s0
                                # elif zero == 'y': good_x = s1
                                # elif zero == '-y':
                                #     if s1[0] == '-': good_x = s1[-1]
                                #     else: good_x = '-'+s1
                                # elif zero == 'z': good_x = s2
                                # elif zero == '-z':
                                #     if s2[0] == '-': good_x = s2[-1]
                                #     else: good_x = '-'+s2

                                # if one == 'x': good_y = s0
                                # elif one == '-x':
                                #     if s0[0] == '-': good_y = s0[-1]
                                #     else: good_y = '-'+s0
                                # elif one == 'y': good_y = s1
                                # elif one == '-y':
                                #     if s1[0] == '-': good_y = s1[-1]
                                #     else: good_y = '-'+s1
                                # elif one == 'z': good_y = s2
                                # elif one == '-z':
                                #     if s2[0] == '-': good_y = s2[-1]
                                #     else: good_y = '-'+s2
                                
                                # if two == 'x': good_z = s0
                                # elif two == '-x':
                                #     if s0[0] == '-': good_z = s0[-1]
                                #     else: good_z = '-'+s0
                                # elif two == 'y': good_z = s1
                                # elif two == '-y':
                                #     if s1[0] == '-': good_z = s1[-1]
                                #     else: good_z = '-'+s1
                                # elif two == 'z': good_z = s2
                                # elif two == '-z':
                                #     if s2[0] == '-': good_z = s2[-1]
                                #     else: good_z = '-'+s2

                                # good_rotation = good_x + ',' + good_y + ',' + good_z
                                good_rotation = rotation
                                return common_beacons, other_common_beacons, good_rotation
                            break
    return None, None, None            
            

In [299]:
class Scanner:

    def __init__(self, list_of_beacons):
        self.list_of_beacons = list_of_beacons
        self.shared_beacons = {}
        self.zero_pos = (0,0,0)
        with open('19_combinations.txt') as f:
            self.rotations = f.readlines()
        self.rotations = [r.strip() for r in self.rotations]
        self.orientation = 'x,y,z'
        self.set = False
        print("new")
        
    
    def add_beacon(self, beacon):
        self.list_of_beacons.append(beacon)

    def check_overlap(self, other) -> bool:
        common_beacons = []
        other_common_beacons = []
        good_rotation = 'x,y,z'
        for rotation in self.rotations:
            common_beacons = []
            other_common_beacons = []
            for i in range(len(self.list_of_beacons)):
                if len(common_beacons) >= 12: break
                for j in range(i+1,len(self.list_of_beacons)):
                    if len(common_beacons) >= 12: break
                    for k in range(len(other.list_of_beacons)):
                        if len(common_beacons) >= 12: break
                        for l in range(k+1, len(other.list_of_beacons)):
                            # same: whether or not these two pairs of beacons have the same magnitude of difference.
                            same = same_beacons(self.list_of_beacons[i], self.list_of_beacons[j], other.list_of_beacons[k], other.list_of_beacons[l], rotation)
                            if same:
                                if i not in common_beacons: common_beacons.append(i)
                                if j not in common_beacons: common_beacons.append(j)
                                if k not in other_common_beacons: other_common_beacons.append(k)
                                if l not in other_common_beacons: other_common_beacons.append(l)
                                break
            if len(common_beacons) >= 12:
                # figure out the absolute rotation right now ??
                # '-y x -z' base
                # 'x z -y' curr
                # what is the absolute rotation?
                # [0] value -> -y
                # [1] value -> -z
                # [2] value -> -x
                super_rotation = self.orientation
                s0, s1, s2 = super_rotation.split(',')
                zero,one,two = rotation.split(',')

                if zero == 'x': good_x = s0
                elif zero == '-x':
                    if s0[0] == '-': good_x = s0[-1]
                    else: good_x = '-'+s0
                elif zero == 'y': good_x = s1
                elif zero == '-y':
                    if s1[0] == '-': good_x = s1[-1]
                    else: good_x = '-'+s1
                elif zero == 'z': good_x = s2
                elif zero == '-z':
                    if s2[0] == '-': good_x = s2[-1]
                    else: good_x = '-'+s2

                if one == 'x': good_y = s0
                elif one == '-x':
                    if s0[0] == '-': good_y = s0[-1]
                    else: good_y = '-'+s0
                elif one == 'y': good_y = s1
                elif one == '-y':
                    if s1[0] == '-': good_y = s1[-1]
                    else: good_y = '-'+s1
                elif one == 'z': good_y = s2
                elif one == '-z':
                    if s2[0] == '-': good_y = s2[-1]
                    else: good_y = '-'+s2
                
                if two == 'x': good_z = s0
                elif two == '-x':
                    if s0[0] == '-': good_z = s0[-1]
                    else: good_z = '-'+s0
                elif two == 'y': good_z = s1
                elif two == '-y':
                    if s1[0] == '-': good_z = s1[-1]
                    else: good_z = '-'+s1
                elif two == 'z': good_z = s2
                elif two == '-z':
                    if s2[0] == '-': good_z = s2[-1]
                    else: good_z = '-'+s2

                good_rotation = good_x + ',' + good_y + ',' + good_z
                break
        if len(common_beacons) >= 12:
            self.shared_beacons[other] = common_beacons
            other.shared_beacons[self] = other_common_beacons
            other.orientation = good_rotation
        return len(common_beacons) >= 12

    def set_zero_pos(self,coords):
        self.zero_pos = (coords[0],coords[1],coords[2])
        self.set = True
        

In [300]:
# class Scanner:

#     def __init__(self, list_of_beacons):
#         self.list_of_beacons = list_of_beacons
#         self.shared_beacons = {}
#         self.zero_pos = (0,0,0)
#         with open('19_combinations.txt') as f:
#             self.rotations = f.readlines()
#         self.rotations = [r.strip() for r in self.rotations]
#         self.orientation = 'x,y,z'
#         self.set = False
#         self.absolute_beacons = []
#         print("new")
    
#     def add_beacon(self, beacon):
#         self.list_of_beacons.append(beacon)
#         self.absolute_beacons.append(beacon)

#     def check_overlap(self, other) -> bool:
#         common_beacons, other_common_beacons, good_rotation = find_commons(self.rotations, self.absolute_beacons, other.absolute_beacons ,\
#             self.orientation)
#         overlap = not common_beacons is None
#         if overlap:
#             self.shared_beacons[other] = common_beacons
#             other.shared_beacons[self] = other_common_beacons
#             other.orientation = good_rotation
#         return overlap

#     def set_zero_pos(self,coords):
#         self.absolute_beacons = []
#         self.zero_pos = (coords[0],coords[1],coords[2])
#         # for beac in self.list_of_beacons:
#         #     b = rotate_transform(beac, self.orientation)
#         #     absolute_b0 = int(b[0]) + int(self.zero_pos[0])
#         #     absolute_b1 = int(b[1]) + int(self.zero_pos[1])
#         #     absolute_b2 = int(b[2]) + int(self.zero_pos[2])
#         #     self.absolute_beacons.append((absolute_b0,absolute_b1,absolute_b2))
#         self.set = True
#         # self.orientation = 'x,y,z'
    
        

In [301]:
scanners = []
for line in lines:
    if len(line) > 0 and line[1] == '-':
        curr_scanner = Scanner([])
        scanners.append(curr_scanner)
    elif len(line) > 1:
        coords = line.split(',')
        curr_scanner.add_beacon((int(coords[0]), int(coords[1]), int(coords[2])))
        # print(curr_scanner.list_of_beacons)

new
new
new
new
new


In [302]:
def same_sign(num1,num2):
    return copysign(1,num1) == copysign(1,num2)

In [303]:
def find_scanner_pos(scanner1: Scanner, scanner2: Scanner) -> Tuple[int, int, int]:
    # we either add or subtract each of the coords.
    print('orientation 1: ' + scanner1.orientation)
    print('orientation 2: ' + scanner2.orientation)
    orientation1 = scanner1.orientation.split(',')
    absolute_orientation1 = (orientation1[0][-1], orientation1[1][-1], orientation1[2][-1])
    if orientation1[0] == 'x' or orientation1[0] == '-x': xi1 = 0
    elif orientation1[0] == 'y' or orientation1[0] == '-y': yi1 = 0
    elif orientation1[0] == 'z' or orientation1[0] =='-z': zi1 = 0
    else: print("o0")
    
    if orientation1[1] == 'x' or orientation1[1] == '-x': xi1 = 1
    elif orientation1[1] == 'y' or orientation1[1] == '-y': yi1 = 1
    elif orientation1[1] == 'z' or orientation1[1] == '-z': zi1 = 1
    else: print('o1')

    if orientation1[2] == 'x' or orientation1[2] == '-x': xi1 = 2
    elif orientation1[2] == 'y' or orientation1[2] == '-y': yi1 = 2
    elif orientation1[2] == 'z' or orientation1[2] == '-z': zi1 = 2
    else: print('o2')

    x1sgn = -1 if orientation1[xi1][0] == '-' else 1
    y1sgn = -1 if orientation1[yi1][0] == '-' else 1
    z1sgn = -1 if orientation1[zi1][0] == '-' else 1

    orientation2 = scanner2.orientation.split(',')
    # absolute_orientation2 = (orientation2[0][-1], orientation2[1][-1], orientation2[2][-1])
    if orientation2[0] == 'x' or orientation2[0] == '-x': xi2 = 0
    elif orientation2[0] == 'y' or orientation2[0] == '-y': yi2 = 0
    elif orientation2[0] == 'z' or orientation2[0] =='-z': zi2 = 0
    
    if orientation2[1] == 'x' or orientation2[1] == '-x': xi2 = 1
    elif orientation2[1] == 'y' or orientation2[1] == '-y': yi2 = 1
    elif orientation2[1] == 'z' or orientation2[1] == '-z': zi2 = 1
    else: print('o1')

    if orientation2[2] == 'x' or orientation2[2] == '-x': xi2 = 2
    elif orientation2[2] == 'y' or orientation2[2] == '-y': yi2 = 2
    elif orientation2[2] == 'z' or orientation2[2] == '-z': zi2 = 2
    else: print('o2')
    # if absolute_orientation2[0] == absolute_orientation1[xi1]: xi2 = 0
    # elif absolute_orientation2[0] == absolute_orientation1[yi1]: yi2 = 0
    # elif absolute_orientation2[0] == absolute_orientation1[zi1]: zi2 = 0
    
    # if absolute_orientation2[1] == absolute_orientation1[xi1]: xi2 = 1
    # elif absolute_orientation2[1] == absolute_orientation1[yi1]: yi2 = 1
    # elif absolute_orientation2[1] == absolute_orientation1[zi1]: zi2 = 1

    # if absolute_orientation2[2] == absolute_orientation1[xi1]: xi2 = 2
    # elif absolute_orientation2[2] == absolute_orientation1[yi1]: yi2 = 2
    # elif absolute_orientation2[2] == absolute_orientation1[zi1]: zi2 = 2

    x2sgn = -1 if orientation2[xi2][0] == '-' else 1
    y2sgn = -1 if orientation2[yi2][0] == '-' else 1
    z2sgn = -1 if orientation2[zi2][0] == '-' else 1

    beacon_ls1 = scanner1.list_of_beacons
    beacon_ls2 = scanner2.list_of_beacons
    shared_indices1 = scanner1.shared_beacons[scanner2]
    shared_indices2 = scanner2.shared_beacons[scanner1]
    shared_coords1 = [beacon_ls1[x] for x in shared_indices1]
    shared_coords2 = [beacon_ls2[x] for x in shared_indices2]

    x = shared_coords1[0][xi1] * x1sgn - shared_coords2[0][xi2] * x2sgn  + scanner1.zero_pos[0]
    y = shared_coords1[0][yi1] * y1sgn - shared_coords2[0][yi2] * y2sgn + scanner1.zero_pos[1]
    z = shared_coords1[0][zi1] * z1sgn - shared_coords2[0][zi2] * z2sgn  + scanner1.zero_pos[2] 
    
    return (x,y,z)

In [304]:
# def find_scanner_pos(scanner1: Scanner, scanner2: Scanner) -> Tuple[int, int, int]:
#     # we either add or subtract each of the coords.
#     print('orientation 1: ' + scanner1.orientation)
#     print('orientation 2: ' + scanner2.orientation)
#     orientation1 = scanner1.orientation.split(',')
#     absolute_orientation1 = (orientation1[0][-1], orientation1[1][-1], orientation1[2][-1])
#     if orientation1[0] == 'x' or orientation1[0] == '-x': xi1 = 0
#     elif orientation1[0] == 'y' or orientation1[0] == '-y': yi1 = 0
#     elif orientation1[0] == 'z' or orientation1[0] =='-z': zi1 = 0
#     else: print("o0")
    
#     if orientation1[1] == 'x' or orientation1[1] == '-x': xi1 = 1
#     elif orientation1[1] == 'y' or orientation1[1] == '-y': yi1 = 1
#     elif orientation1[1] == 'z' or orientation1[1] == '-z': zi1 = 1
#     else: print('o1')

#     if orientation1[2] == 'x' or orientation1[2] == '-x': xi1 = 2
#     elif orientation1[2] == 'y' or orientation1[2] == '-y': yi1 = 2
#     elif orientation1[2] == 'z' or orientation1[2] == '-z': zi1 = 2
#     else: print('o2')

#     x1sgn = -1 if orientation1[xi1][0] == '-' else 1
#     y1sgn = -1 if orientation1[yi1][0] == '-' else 1
#     z1sgn = -1 if orientation1[zi1][0] == '-' else 1

#     orientation2 = scanner2.orientation.split(',')
#     # absolute_orientation2 = (orientation2[0][-1], orientation2[1][-1], orientation2[2][-1])
#     if orientation2[0] == 'x' or orientation2[0] == '-x': xi2 = 0
#     elif orientation2[0] == 'y' or orientation2[0] == '-y': yi2 = 0
#     elif orientation2[0] == 'z' or orientation2[0] =='-z': zi2 = 0
    
#     if orientation2[1] == 'x' or orientation2[1] == '-x': xi2 = 1
#     elif orientation2[1] == 'y' or orientation2[1] == '-y': yi2 = 1
#     elif orientation2[1] == 'z' or orientation2[1] == '-z': zi2 = 1
#     else: print('o1')

#     if orientation2[2] == 'x' or orientation2[2] == '-x': xi2 = 2
#     elif orientation2[2] == 'y' or orientation2[2] == '-y': yi2 = 2
#     elif orientation2[2] == 'z' or orientation2[2] == '-z': zi2 = 2
#     else: print('o2')
#     # if absolute_orientation2[0] == absolute_orientation1[xi1]: xi2 = 0
#     # elif absolute_orientation2[0] == absolute_orientation1[yi1]: yi2 = 0
#     # elif absolute_orientation2[0] == absolute_orientation1[zi1]: zi2 = 0
    
#     # if absolute_orientation2[1] == absolute_orientation1[xi1]: xi2 = 1
#     # elif absolute_orientation2[1] == absolute_orientation1[yi1]: yi2 = 1
#     # elif absolute_orientation2[1] == absolute_orientation1[zi1]: zi2 = 1

#     # if absolute_orientation2[2] == absolute_orientation1[xi1]: xi2 = 2
#     # elif absolute_orientation2[2] == absolute_orientation1[yi1]: yi2 = 2
#     # elif absolute_orientation2[2] == absolute_orientation1[zi1]: zi2 = 2

#     x2sgn = -1 if orientation2[xi2][0] == '-' else 1
#     y2sgn = -1 if orientation2[yi2][0] == '-' else 1
#     z2sgn = -1 if orientation2[zi2][0] == '-' else 1

#     beacon_ls1 = scanner1.absolute_beacons
#     beacon_ls2 = scanner2.absolute_beacons
#     shared_indices1 = scanner1.shared_beacons[scanner2]
#     shared_indices2 = scanner2.shared_beacons[scanner1]
#     shared_coords1 = [beacon_ls1[x] for x in shared_indices1]
#     shared_coords2 = [beacon_ls2[x] for x in shared_indices2]

#     x = shared_coords1[0][xi1] * x1sgn - shared_coords2[0][xi2] * x2sgn  + scanner1.zero_pos[0]
#     y = shared_coords1[0][yi1] * y1sgn - shared_coords2[0][yi2] * y2sgn + scanner1.zero_pos[1]
#     z = shared_coords1[0][zi1] * z1sgn - shared_coords2[0][zi2] * z2sgn  + scanner1.zero_pos[2] 
    
#     return (x,y,z)

In [305]:
with open('19_combinations.txt') as f:
        rotations = f.readlines()
        rotations = [r.strip() for r in rotations]

In [306]:
# global_beacons = set()
# for s in scanners[0].list_of_beacons:
#     global_beacons.add(s)


# for idx in range(1,len(scanners)):
#     curr_scanner = scanners[idx]
#     done = False
#     for rotation in rotations:
#         common_beacons = []
#         for a1 in global_beacons:
#             for a2 in global_beacons:
#                 for i in range(len(curr_scanner.list_of_beacons)):
#                     b1 = rotate_transform(curr_scanner.list_of_beacons[i], rotation)
#                     for j in range(i+1, len(curr_scanner.list_of_beacons)):
#                         b2 = rotate_transform(curr_scanner.list_of_beacons[j], rotation)
#                         same = a1[0]-a2[0] == b1[0]-b2[0] and a1[1]-a2[1] == b1[1]-b2[1] and a1[2]-a2[2] == b1[2]-b2[2]
#                         if same:
#                             if i not in common_beacons: common_beacons.append(i)
#                             if j not in common_beacons: common_beacons.append(j)
#                             if len(common_beacons) >= 12:
#                                 curr_scanner.orientation = rotation
#                                 for k in range(len(curr_scanner.list_of_beacons)):
#                                     if k != i and k != j:
#                                         beac = rotate_transform(curr_scanner.list_of_beacons[k],rotation)
#                                         global_beacons.add(beac)
#                                 done = True
#                                 break
#                     if done: break
#                 if done: break
#             if done: break
                            
                            
                        

        

In [307]:
print(len(global_beacons))

95


In [308]:
scnr0 = scanners[0]
scnr0.set = True
scnr1 = scanners[1]
base_scnr = scnr0
for idx in range(len(scanners)):
    base_scnr = scanners[idx]
    for i in range(len(scanners)):
        if i != idx:
            curr_scnr = scanners[i]
            if not curr_scnr.set and base_scnr.check_overlap(curr_scnr):
                curr_scnr.set_zero_pos(find_scanner_pos(base_scnr, curr_scnr))
                print(base_scnr.zero_pos)
                # print("or1: " + base_scnr.orientation)
                # print("or2: " + curr_scnr.orientation)
                print("from scanner #" + str(idx))
                print("scanner #" + str(i) + "\t" + str(curr_scnr.zero_pos))

orientation 1: x,y,z
orientation 2: -x,y,-z
(0, 0, 0)
from scanner #0
scanner #1	(68, -1246, -43)
orientation 1: -x,y,-z
orientation 2: -x,y,-z
(68, -1246, -43)
from scanner #1
scanner #3	(-92, -2380, -20)
orientation 1: -x,y,-z
orientation 2: z,-x,-y
(68, -1246, -43)
from scanner #1
scanner #4	(-20, -1133, 1061)
orientation 1: z,-x,-y
orientation 2: -x,z,y
(-20, -1133, 1061)
from scanner #4
scanner #2	(1105, -1205, 1229)


In [309]:
beacons_set = set()
for scanner in scanners:
    for beac in scanner.list_of_beacons:
        b = rotate_transform(beac, scanner.orientation)
        absolute_b0 = int(b[0]) + int(scanner.zero_pos[0])
        absolute_b1 = int(b[1]) + int(scanner.zero_pos[1])
        absolute_b2 = int(b[2]) + int(scanner.zero_pos[2])
        beacons_set.add((absolute_b0,absolute_b1,absolute_b2))
        

print(len(beacons_set))
beacons_set

79


{(-892, 524, 684),
 (-876, 649, 763),
 (-838, 591, 734),
 (-789, 900, -551),
 (-739, -1745, 668),
 (-706, -3180, -659),
 (-697, -3072, -689),
 (-689, 845, -530),
 (-687, -1600, 576),
 (-661, -816, -575),
 (-654, -3158, -753),
 (-635, -1737, 486),
 (-631, -672, 1502),
 (-624, -1620, 1868),
 (-620, -3212, 371),
 (-618, -824, -621),
 (-612, -1695, 1788),
 (-601, -1648, -643),
 (-584, 868, -557),
 (-537, -823, -458),
 (-532, -1715, 1894),
 (-518, -1681, -600),
 (-499, -1607, -770),
 (-485, -357, 347),
 (-470, -3283, 303),
 (-456, -621, 1527),
 (-447, -329, 318),
 (-430, -3130, 366),
 (-413, -627, 1469),
 (-345, -311, 381),
 (-36, -1284, 1171),
 (-27, -1108, -65),
 (7, -33, -71),
 (12, -2351, -103),
 (26, -1119, 1091),
 (346, -2985, 342),
 (366, -3059, 397),
 (377, -2827, 367),
 (390, -675, -793),
 (396, -1931, -563),
 (404, -588, -901),
 (408, -1815, 803),
 (423, -701, 434),
 (432, -2009, 850),
 (443, 580, 662),
 (455, 729, 728),
 (456, -540, 1869),
 (459, -707, 401),
 (465, -695, 1988),
 